<a href="https://colab.research.google.com/github/vraghavans1/mlp-project/blob/main/Copy_of_22f3002203_notebook_t22025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

engage_2_value_from_clicks_to_conversions_path = kagglehub.competition_download('engage-2-value-from-clicks-to-conversions')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.dummy import DummyRegressor
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## DATA LOADING  

In [ ]:
train_data=pd.read_csv('train_data.csv')
test_data=pd.read_csv('test_data.csv')
sample_data=pd.read_csv('sample_submission.csv')

In [ ]:
train_data.head()

,trafficSource.isTrueDirect,purchaseValue,browser,device.screenResolution,trafficSource.adContent,trafficSource.keyword,screenSize,geoCluster,trafficSource.adwordsClickInfo.slot,device.mobileDeviceBranding,...,device.language,deviceType,userChannel,device.browserVersion,totalHits,device.screenColors,sessionStart,geoNetwork.continent,device.isMobile,new_visits
0,NaN,0.0,Edge,not available in demo dataset,NaN,NaN,medium,Region_2,NaN,not available in demo dataset,...,not available in demo dataset,desktop,Social,not available in demo dataset,1.0,not available in demo dataset,1.500101e+09,Americas,False,1.0
1,True,0.0,Chrome,not available in demo dataset,NaN,NaN,medium,Region_3,NaN,not available in demo dataset,...,not available in demo dataset,desktop,Direct,not available in demo dataset,1.0,not available in demo dataset,1.495262e+09,Americas,False,1.0
2,True,0.0,Chrome,not available in demo dataset,NaN,(not provided),medium,Region_2,NaN,not available in demo dataset,...,not available in demo dataset,desktop,Organic Search,not available in demo dataset,6.0,not available in demo dataset,1.508510e+09,Europe,False,NaN
3,NaN,0.0,Internet Explorer,not available in demo dataset,NaN,NaN,medium,Region_4,NaN,not available in demo dataset,...,not available in demo dataset,desktop,Social,not available in demo dataset,1.0,not available in demo dataset,1.483432e+09,Asia,False,1.0
4,True,88950000.0,Chrome,not available in demo dataset,NaN,NaN,medium,Region_3,NaN,not available in demo dataset,...,not available in demo dataset,desktop,Direct,not available in demo dataset,66.0,not available in demo dataset,1.475805e+09,Americas,False,1.0


In [ ]:
test_data.head()

##  EXPLORATORY DATA ANALYSIS

In [ ]:
train_data.shape, test_data.shape

Summary of the Data set

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108350 entries, 0 to 108349
Data columns (total 52 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   trafficSource.isTrueDirect                    40093 non-null   object 
 1   purchaseValue                                 108350 non-null  float64
 2   browser                                       108350 non-null  object 
 3   device.screenResolution                       108350 non-null  object 
 4   trafficSource.adContent                       2750 non-null    object 
 5   trafficSource.keyword                         41144 non-null   object 
 6   screenSize                                    108350 non-null  object 
 7   geoCluster                                    108350 non-null  object 
 8   trafficSource.adwordsClickInfo.slot           3979 non-null    object 
 9   device.mobileDeviceBranding                   10

In [ ]:
test_data.info()

### Identify missing values that may require imputation or removal.

In [ ]:
missing_values = train_data.isnull().sum().sort_values(ascending=False)
print(missing_values[missing_values > 0])

In [ ]:
# Calculate and sort the percentage of missing values per feature in descending order
missing_percentage = (train_data.isnull().sum() / len(train_data)) * 100
missing_percentage = missing_percentage[missing_percentage > 0].sort_values(ascending=False)
# print(missing_percentage)

plt.figure(figsize=(10, 6))
missing_percentage.plot(kind='bar', color='red')
plt.title('Percentage of Missing Values per Feature')
plt.xlabel('Features')
plt.ylabel('Missing Value Percentage')
plt.show()


### Null values in the Test data

In [ ]:
display(test_data[test_data.columns[test_data.isnull().any()]].isnull().sum())

In [ ]:
test_null_columns = set(test_data.columns[test_data.isnull().any()])
train_null_columns = set(train_data.columns[train_data.isnull().any()])

common_missing_columns = test_null_columns & train_null_columns

test_only_missing_columns = test_null_columns - train_null_columns
train_only_missing_columns = train_null_columns - test_null_columns

print("Columns with missing values in BOTH train and test data:")
print(common_missing_columns)

print("\nColumns with missing values ONLY in test data:")
print(test_only_missing_columns)

print("\nColumns with missing values ONLY in train data:")
print(train_only_missing_columns)


#### All of the missing values are common to both training and test data

### Unique values in each column in the training data

In [ ]:
train_data.nunique().sort_values(ascending=False)

In [ ]:
top_10_cardinal_columns = train_data.nunique().sort_values(ascending=False).head(10)

plt.figure(figsize=(12, 6))
top_10_cardinal_columns.plot(kind='bar', color='royalblue')

# Customize plot
plt.title("Top 10 High-Cardinality Columns", fontsize=14)
plt.xlabel("Columns", fontsize=12)
plt.ylabel("Unique Values Count", fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()

What is the correlation between the number of pageviews and the purchase value, considering only users who did not bounce?

In [ ]:
# Filter out rows where purchase_value is 0
filtered_data = train_data[train_data['purchaseValue'] != 0].copy()

# Calculate the correlation
correlation = filtered_data['pageViews'].corr(filtered_data['purchaseValue'])

correlation

np.float64(0.12191709637943975)

In [ ]:
# Filter data to include only users who purchased (purchaseValue > 0)
purchased_users_data = train_data[train_data['purchaseValue'] > 0].copy()

# Calculate the correlation between pageviews and purchase value for purchased users
correlation_purchased = purchased_users_data['pageViews'].corr(purchased_users_data['purchaseValue'])
correlation_purchased

np.float64(0.12191709637943975)

Among users from different continents, which continent has the highest average 'totalHits'?

In [ ]:
average_hits_by_continent = train_data.groupby('geoNetwork.continent')['totalHits'].mean()

# Find the continent with the highest average totalHits
continent_highest_hits = average_hits_by_continent.idxmax()
highest_average_hits = average_hits_by_continent.max()

print(f"The continent with the highest average 'totalHits' is: {continent_highest_hits}")

The continent with the highest average 'totalHits' is: Americas


For users who made a purchase, what is the most common traffic source (medium)?

In [ ]:
# Filter data for rows where purchaseValue is not 0
purchased_data = train_data[train_data['purchaseValue'] != 0]
# Find the most frequent traffic source medium in the filtered data
most_frequent_medium = purchased_data['trafficSource.medium'].mode()[0]
print(f"The most frequent trafficSource.medium where purchase value is not 0 is: {most_frequent_medium}")

The most frequent trafficSource.medium where purchase value is not 0 is: (none)


Which combination of operating system and browser is most common among users who made a purchase?

In [ ]:

# Filter data for rows where purchaseValue is not 0
purchased_data = train_data[train_data['purchaseValue'] != 0].copy()

# Combine 'device.deviceCategory' and 'device.browser' into a new column
purchased_data['device_browser_combination'] = purchased_data['os'] + ' - ' + purchased_data['browser']

# Find the most frequent combination
most_common_combination = purchased_data['device_browser_combination'].mode()[0]

print(f"The most common combination of deviceType and browser among users who made a purchase is: {most_common_combination}")

The most common combination of deviceType and browser among users who made a purchase is: Macintosh - Chrome


What is the ratio of average number of hits for sessions that resulted in a purchase compared to those that did not?

In [ ]:
# ✅ Ensure correct dtype
train_data["totalHits"] = pd.to_numeric(train_data["totalHits"], errors="coerce")
train_data["purchaseValue"] = pd.to_numeric(train_data["purchaseValue"], errors="coerce")

# 🧮 Split into two groups
hits_with_purchase = train_data[train_data["purchaseValue"] > 0]["totalHits"]
hits_without_purchase = train_data[train_data["purchaseValue"] == 0]["totalHits"]

# 📊 Compute averages
avg_hits_with = hits_with_purchase.mean()
avg_hits_without = hits_without_purchase.mean()

# 🔁 Compute ratio
ratio = avg_hits_with / avg_hits_without

# 🖨️ Display
print(f"✅ Average hits WITH purchase     : {avg_hits_with:.2f}")
print(f"✅ Average hits WITHOUT purchase  : {avg_hits_without:.2f}")
print(f"🔁 Ratio (with / without)         : {ratio:.2f}")


✅ Average hits WITH purchase     : 35.53
✅ Average hits WITHOUT purchase  : 4.14
🔁 Ratio (with / without)         : 8.58


What is the proportion of 'organic' sessions in comparison to all other types of sessions from each traffic source medium?

In [ ]:
organic_sessions = (train_data["trafficSource.medium"] == "organic").sum()
total_sessions = len(train_data)
organic_proportion = organic_sessions / total_sessions
print(f"{organic_proportion:.2f}")

0.35


Q7: Which referral path is most common for sessions that resulted in a purchase?

In [ ]:
# Filter users who made a purchase
purchased = train_data[train_data["purchaseValue"] > 0]

# Most common referral path among purchasers
most_common_referral = purchased["trafficSource.referralPath"].value_counts().idxmax()
count = purchased["trafficSource.referralPath"].value_counts().max()

print(f"🔗 Most common referral path for purchasers: '{most_common_referral}'")
print(f"📊 Number of sessions: {count}")


🔗 Most common referral path for purchasers: '/'
📊 Number of sessions: 5044


Q8: Which operating system has the highest average purchase value?

In [ ]:
# Group by OS and calculate average purchase value
os_avg_purchase = train_data.groupby("os")["purchaseValue"].mean().sort_values(ascending=False)

# Get top OS
top_os = os_avg_purchase.idxmax()
top_value = os_avg_purchase.max()

print("💻 OS with highest avg. purchaseValue:", top_os)
print("💰 Average purchaseValue:", round(top_value, 2))


💻 OS with highest avg. purchaseValue: Chrome OS
💰 Average purchaseValue: 81268980.86


Q9: What is the average number of sessions per user?

In [ ]:
# Convert sessionNumber to numeric if needed
train_data["sessionNumber"] = pd.to_numeric(train_data["sessionNumber"], errors="coerce")

# Group by userId and calculate average sessions
avg_sessions_per_user = train_data.groupby("userId")["sessionNumber"].mean().mean()

print(f"📊 Average number of sessions per user: {avg_sessions_per_user:.2f}")


📊 Average number of sessions per user: 1.98


Q10: How many unique users are there in the dataset?

In [ ]:
# Count unique users
unique_users = train_data["userId"].nunique()

print(f"🙋 Unique users in the dataset: {unique_users}")


🙋 Unique users in the dataset: 100499


## MODELS

In [ ]:
# X = train_data.drop(columns='purchaseValue')
# y = train_data['purchaseValue']
# test_data = test_data

# # Make object columns categorical in both train and test
# object_columns = X.select_dtypes(include='object').columns
# for col in object_columns:
#     X[col] = X[col].astype('category')
#     if col in test_data.columns:
#         test_data[col] = test_data[col].astype('category')

# X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)
# model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05, random_state=42)
# model.fit(X_train, y_train)

# # Make predictions
# y_pred = model.predict(test_data)

# # Create submission DataFrame
# submission = pd.DataFrame({
#     'id':range(0,test_data.shape[0]),
#     'purchaseValue': y_pred
# })

# # Save to CSV
# submission.to_csv('submission.csv', index=False)


In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

# # Split into features and target
# X = train_data.drop(columns=['purchaseValue'])
# y = train_data['purchaseValue']

# # Handling missing values and encoding categorical columns
# numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
# categorical_features = X.select_dtypes(include=['object']).columns

# # Preprocessing pipelines
# numeric_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='mean')),  # Handle missing values
#     ('scaler', StandardScaler())  # Standardize numerical data
# ])

# categorical_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
#     ('encoder', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding for categorical features
# ])

# # Apply transformations to columns
# preprocessor = ColumnTransformer([
#     ('num', numeric_pipeline, numeric_features),
#     ('cat', categorical_pipeline, categorical_features)
# ])

# # Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import r2_score

# # Define a smaller and more efficient hyperparameter space
# param_distributions = {
#     'regressor__n_estimators': [50, 100],
#     'regressor__max_depth': [10, 20, None],
#     'regressor__min_samples_split': [2, 5],
# }

# # Build the model pipeline again if needed
# model_pipeline = Pipeline([
#     ('preprocessor', preprocessor),  # Your earlier preprocessing pipeline
#     ('regressor', RandomForestRegressor(random_state=42))
# ])

# # Setup RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     model_pipeline,
#     param_distributions=param_distributions,
#     n_iter=6,                # Try 6 random combinations (faster)
#     cv=3,                    # Use 3-fold cross-validation
#     scoring='neg_mean_squared_error',
#     n_jobs=-1,               # Use all available cores
#     verbose=1,
#     random_state=42
# )

# # Fit on a subset (optional but can improve speed during testing)
# X_sample = X_train
# y_sample = y_train.loc[X_sample.index]

# # Fit the model
# random_search.fit(X_sample, y_sample)

# # Best model
# best_model = random_search.best_estimator_

# # Evaluate on full validation set
# y_pred_best = best_model.predict(X_val)
# r2_best = r2_score(y_val,y_pred_best)
# print(f'Best R² Score: {r2_best:.4f}')

# # Create submission DataFrame
# submission = pd.DataFrame({
#     'id':range(0,test_data.shape[0]),
#     'purchaseValue': y_pred_best
# })

# # Save to CSV
# submission.to_csv('submission.csv', index=False)



In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.metrics import r2_score
# import lightgbm as lgb

# # -------------------------------
# # 1. Load your dataset
# # -------------------------------
# # Assume you already have train_data and test_data loaded
# # train_data = pd.read_csv('train.csv')
# # test_data = pd.read_csv('test.csv')

# # -------------------------------
# # 2. Prepare features and target
# # -------------------------------
# X = train_data.drop(columns=['purchaseValue'])
# y = train_data['purchaseValue']

# # Optional: log1p transform to handle skewed targets
# y_log = np.log1p(y)

# X_train, X_val, y_train, y_val = train_test_split(X, y_log, test_size=0.2, random_state=42)

# # -------------------------------
# # 3. Build preprocessing pipeline
# # -------------------------------
# numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
# categorical_features = X.select_dtypes(include=['object', 'category']).columns

# numeric_transformer = Pipeline([
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# categorical_transformer = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
# ])

# preprocessor = ColumnTransformer([
#     ('num', numeric_transformer, numeric_features),
#     ('cat', categorical_transformer, categorical_features)
# ])

# # -------------------------------
# # 4. Define the LightGBM model
# # -------------------------------
# lgb_model = lgb.LGBMRegressor(random_state=42)

# # -------------------------------
# # 5. Create the pipeline
# # -------------------------------
# pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('regressor', lgb_model)
# ])

# # -------------------------------
# # 6. Hyperparameter tuning
# # -------------------------------
# param_distributions = {
#     'regressor__n_estimators': [100, 200, 300],
#     'regressor__learning_rate': [0.01, 0.05, 0.1],
#     'regressor__max_depth': [5, 10, -1],
#     'regressor__num_leaves': [15, 31, 63]
# }

# search = RandomizedSearchCV(
#     pipeline,
#     param_distributions=param_distributions,
#     n_iter=10,
#     scoring='r2',
#     cv=3,
#     verbose=1,
#     n_jobs=-1,
#     random_state=42
# )

# search.fit(X_train, y_train)

# # -------------------------------
# # 7. Evaluate
# # -------------------------------
# best_model = search.best_estimator_
# y_val_pred_log = best_model.predict(X_val)
# y_val_pred = np.expm1(y_val_pred_log)  # Inverse log1p
# y_val_true = np.expm1(y_val)           # Inverse log1p

# r2 = r2_score(y_val_true, y_val_pred)
# print(f'Validation R² score: {r2:.4f}')

# # -------------------------------
# # 8. Predict on test set
# # -------------------------------
# X_test = test_data
# test_preds_log = best_model.predict(X_test)
# #test_preds = np.expm1(test_preds_log)

# submission = pd.DataFrame({
#     'id':range(0,test_data.shape[0]),
#     'purchaseValue': test_preds_log
# })
# submission.to_csv('submission.csv', index=False)


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.impute import SimpleImputer
# from sklearn.metrics import r2_score
# import xgboost as xgb

# # Load data
# train_data = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv")

# # Feature engineering: create a new feature combining device type and browser
# def add_features(df):
#     df['device_browser'] = df['deviceType'] + "_" + df['browser']
#     return df

# train_data = add_features(train_data)

# # Drop columns that are clearly identifiers or too sparse for this context
# drop_cols = ['userId', 'sessionId', 'date', 'sessionStart']
# train_data = train_data.drop(columns=drop_cols, errors='ignore')

# # Separate features and target
# X = train_data.drop(columns=['purchaseValue'])
# y = train_data['purchaseValue']

# # Log transform to reduce skew
# y_log = np.log1p(y)

# # Split for validation
# X_train, X_val, y_train, y_val = train_test_split(X, y_log, test_size=0.2, random_state=42)

# # Column types
# numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
# categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()

# # Pipelines for preprocessing
# numeric_transformer = Pipeline([
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# categorical_transformer = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
# ])

# preprocessor = ColumnTransformer([
#     ('num', numeric_transformer, numeric_features),
#     ('cat', categorical_transformer, categorical_features)
# ])

# # XGBoost model
# xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# # Full pipeline
# pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('regressor', xgb_model)
# ])

# # Basic hyperparameter tuning
# param_distributions = {
#     'regressor__n_estimators': [100, 200],
#     'regressor__max_depth': [3, 5, 7],
#     'regressor__learning_rate': [0.05, 0.1]
# }

# search = RandomizedSearchCV(
#     pipeline,
#     param_distributions=param_distributions,
#     n_iter=6,
#     scoring='r2',
#     cv=3,
#     verbose=1,
#     random_state=42,
#     n_jobs=-1
# )

# # Train the model
# search.fit(X_train, y_train)

# # Evaluate
# best_model = search.best_estimator_
# y_val_pred_log = best_model.predict(X_val)
# y_val_pred = np.expm1(y_val_pred_log)
# y_val_true = np.expm1(y_val)
# r2 = r2_score(y_val_true, y_val_pred)
# print(f'Validation R² score: {r2:.4f}')

# test_data = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv")
# test_data = add_features(test_data)
# test_data = test_data.drop(columns=drop_cols, errors='ignore')
# test_preds_log = best_model.predict(test_data)
# test_preds = np.expm1(test_preds_log)
# submission = pd.DataFrame({'id':range(0,test_data.shape[0]), 'purchaseValue': test_preds})
# submission.to_csv("submission.csv", index=False)


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.impute import SimpleImputer
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import r2_score
# from xgboost import XGBRegressor
# import matplotlib.pyplot as plt

# # -----------------------------
# # Step 1: Load and prepare data
# # -----------------------------
# train_data = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv")

# # Drop ID/session/timestamp columns
# drop_cols = ["userId", "sessionId", "date", "sessionStart"]
# X = train_data.drop(columns=["purchaseValue"] + drop_cols, errors='ignore')
# y = train_data["purchaseValue"]
# y_log = np.log1p(y)

# # Split data
# X_train, X_val, y_train, y_val = train_test_split(X, y_log, test_size=0.2, random_state=42)

# # Identify feature types
# numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
# categorical_features = X.select_dtypes(include=["object", "bool"]).columns.tolist()

# print(f"📊 Original features: {len(numeric_features)} numeric, {len(categorical_features)} categorical")

# # -----------------------------
# # Step 2: Preprocessing
# # -----------------------------
# numeric_transformer = Pipeline([
#     ("imputer", SimpleImputer(strategy="mean")),
#     ("scaler", StandardScaler())
# ])

# categorical_transformer = Pipeline([
#     ("imputer", SimpleImputer(strategy="most_frequent")),
#     ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
# ])

# preprocessor = ColumnTransformer([
#     ("num", numeric_transformer, numeric_features),
#     ("cat", categorical_transformer, categorical_features)
# ])

# # -----------------------------
# # Step 3: RandomForest for Feature Importance
# # -----------------------------
# rf_pipeline = Pipeline([
#     ("preprocessor", preprocessor),
#     ("regressor", RandomForestRegressor(random_state=42))
# ])

# rf_pipeline.fit(X_train, y_train)

# # Get top features
# feature_names = rf_pipeline.named_steps["preprocessor"].get_feature_names_out()
# importances = rf_pipeline.named_steps["regressor"].feature_importances_

# feat_df = pd.DataFrame({"feature": feature_names, "importance": importances})
# feat_df = feat_df.sort_values(by="importance", ascending=False)

# print(f"🔍 Total transformed features: {len(feature_names)}")
# print(f"📈 Max importance: {feat_df['importance'].max():.6f}")
# print(f"📉 Min importance: {feat_df['importance'].min():.6f}")

# # Keep features with importance > 0.001
# important_features = feat_df[feat_df["importance"] > 0.001]["feature"].tolist()
# print(f"🎯 Features with importance > 0.001: {len(important_features)}")

# # Extract original feature names from transformed feature names
# original_feature_names = []
# for feat in important_features:
#     # Handle both numeric and categorical transformed names
#     if "__" in feat:
#         original_name = feat.split("__")[1]  # For categorical: "cat__feature_name"
#     else:
#         original_name = feat.replace("num__", "")  # For numeric: "num__feature_name"

#     if original_name not in original_feature_names:
#         original_feature_names.append(original_name)

# print(f"🔧 Mapped to {len(original_feature_names)} original features")

# # 🛡️ Enhanced Fallback: If no features remain or mapping failed, keep all
# if len(original_feature_names) == 0:
#     print("⚠️ No features passed importance threshold or mapping failed. Using all features.")
#     original_feature_names = numeric_features + categorical_features

# # Ensure we have valid features
# reduced_numeric = [f for f in original_feature_names if f in numeric_features]
# reduced_categorical = [f for f in original_feature_names if f in categorical_features]

# print(f"✅ Final selection: {len(reduced_numeric)} numeric, {len(reduced_categorical)} categorical")

# # Additional safety check
# if len(reduced_numeric) == 0 and len(reduced_categorical) == 0:
#     print("🚨 Emergency fallback: Using all original features")
#     reduced_numeric = numeric_features
#     reduced_categorical = categorical_features

# # -----------------------------
# # Step 4: XGBoost Pipeline
# # -----------------------------
# reduced_preprocessor = ColumnTransformer([
#     ("num", numeric_transformer, reduced_numeric),
#     ("cat", categorical_transformer, reduced_categorical)
# ])

# xgb_pipeline = Pipeline([
#     ("preprocessor", reduced_preprocessor),
#     ("regressor", XGBRegressor(objective="reg:squarederror", random_state=42))
# ])

# # Debug: Check preprocessor output shape
# print("🔬 Testing preprocessor...")
# X_train_transformed = reduced_preprocessor.fit_transform(X_train)
# print(f"📐 Transformed training data shape: {X_train_transformed.shape}")

# if X_train_transformed.shape[1] == 0:
#     raise ValueError("❌ Preprocessor produced 0 features! Check feature selection logic.")

# # Fit model
# print("🚀 Training XGBoost model...")
# xgb_pipeline.fit(X_train, y_train)

# # -----------------------------
# # Step 5: Evaluate
# # -----------------------------
# y_val_pred_log = xgb_pipeline.predict(X_val)
# y_val_pred = np.expm1(y_val_pred_log)
# y_val_true = np.expm1(y_val)
# r2 = r2_score(y_val_true, y_val_pred)
# print(f"✅ Validation R² Score: {r2:.4f}")

# # -----------------------------
# # Step 6: Predict on Test Set
# # -----------------------------
# test_data = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv")
# test_data = test_data.drop(columns=drop_cols, errors='ignore')

# # Predict
# test_preds_log = xgb_pipeline.predict(test_data)
# test_preds = np.expm1(test_preds_log)

# # -----------------------------
# # Step 7: Save submission.csv
# # -----------------------------
# submission = pd.DataFrame({
#     "id": range(0, test_data.shape[0]),
#     "purchaseValue": test_preds
# })
# submission.to_csv("submission.csv", index=False)
# print("📁 submission.csv saved successfully.")

# # Show top selected features for debugging
# print("\n🏆 Top 10 selected features:")
# selected_features_df = feat_df[feat_df["feature"].apply(
#     lambda x: (x.split("__")[1] if "__" in x else x.replace("num__", "")) in original_feature_names
# )].head(10)
# for _, row in selected_features_df.iterrows():
#     print(f"  {row['feature']}: {row['importance']:.6f}")

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split, cross_val_score, KFold
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, PolynomialFeatures
# from sklearn.impute import SimpleImputer
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.linear_model import Ridge, ElasticNet
# from sklearn.metrics import r2_score, mean_squared_error
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# import matplotlib.pyplot as plt
# import seaborn as sns
# import warnings
# warnings.filterwarnings('ignore')

# # -----------------------------
# # Step 1: Load and Advanced EDA
# # -----------------------------
# print("🔄 Loading and exploring data...")
# train_data = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv")
# test_data = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv")

# print(f"📊 Train shape: {train_data.shape}, Test shape: {test_data.shape}")
# print(f"💰 Target stats - Mean: {train_data['purchaseValue'].mean():.2f}, Std: {train_data['purchaseValue'].std():.2f}")
# print(f"🎯 Zero purchases: {(train_data['purchaseValue'] == 0).sum()} ({(train_data['purchaseValue'] == 0).mean()*100:.1f}%)")

# # -----------------------------
# # Step 2: Advanced Feature Engineering
# # -----------------------------
# def advanced_feature_engineering(df, is_train=True):
#     """Create advanced features from raw data"""
#     df_fe = df.copy()

#     # First, standardize data types to prevent mixed type errors
#     print("🔧 Standardizing data types...")

#     # Convert all object and boolean columns to string first
#     for col in df_fe.columns:
#         if df_fe[col].dtype == 'object' or df_fe[col].dtype == 'bool':
#             df_fe[col] = df_fe[col].astype(str)

#     # Date/Time features if available
#     if 'date' in df_fe.columns:
#         try:
#             df_fe['date'] = pd.to_datetime(df_fe['date'])
#             df_fe['dayOfWeek'] = df_fe['date'].dt.dayofweek
#             df_fe['hour'] = df_fe['date'].dt.hour
#             df_fe['month'] = df_fe['date'].dt.month
#             df_fe['isWeekend'] = (df_fe['dayOfWeek'] >= 5).astype(int)
#             df_fe['isBusinessHour'] = ((df_fe['hour'] >= 9) & (df_fe['hour'] <= 17)).astype(int)
#         except:
#             print("⚠️ Could not parse date column")

#     if 'sessionStart' in df_fe.columns:
#         try:
#             df_fe['sessionStart'] = pd.to_datetime(df_fe['sessionStart'])
#             df_fe['sessionHour'] = df_fe['sessionStart'].dt.hour
#             df_fe['sessionDayOfWeek'] = df_fe['sessionStart'].dt.dayofweek
#         except:
#             print("⚠️ Could not parse sessionStart column")

#     # Get categorical columns (now all strings)
#     categorical_cols = []
#     for col in df_fe.columns:
#         if df_fe[col].dtype == 'object' and col not in ['userId', 'sessionId', 'date', 'sessionStart']:
#             categorical_cols.append(col)

#     # Create interaction features for top categorical pairs (limited to prevent explosion)
#     if len(categorical_cols) >= 2:
#         interaction_count = 0
#         for i, col1 in enumerate(categorical_cols[:3]):  # Limit to prevent explosion
#             for col2 in categorical_cols[i+1:4]:
#                 if col1 != col2 and interaction_count < 5:  # Limit interactions
#                     df_fe[f'{col1}_{col2}_interaction'] = df_fe[col1].astype(str) + '_' + df_fe[col2].astype(str)
#                     interaction_count += 1

#     # Numerical feature engineering
#     numeric_cols = []
#     for col in df_fe.columns:
#         if df_fe[col].dtype in ['int64', 'float64'] and col not in ['purchaseValue', 'userId', 'sessionId']:
#             numeric_cols.append(col)

#     # Create polynomial and interaction features for important numerics (limited)
#     feature_count = 0
#     for col in numeric_cols[:3]:  # Further limit to prevent too many features
#         if col in df_fe.columns and feature_count < 10:
#             try:
#                 # Square and log transforms with error handling
#                 df_fe[f'{col}_squared'] = df_fe[col] ** 2
#                 df_fe[f'{col}_log1p'] = np.log1p(np.abs(df_fe[col]))
#                 df_fe[f'{col}_sqrt'] = np.sqrt(np.abs(df_fe[col]))
#                 feature_count += 3
#             except:
#                 print(f"⚠️ Could not create polynomial features for {col}")

#     # Binning continuous variables (limited)
#     for col in numeric_cols[:2]:  # Limit binning
#         if col in df_fe.columns and df_fe[col].nunique() > 10:
#             try:
#                 df_fe[f'{col}_binned'] = pd.qcut(df_fe[col], q=5, labels=False, duplicates='drop')
#             except:
#                 try:
#                     df_fe[f'{col}_binned'] = pd.cut(df_fe[col], bins=5, labels=False, duplicates='drop')
#                 except:
#                     print(f"⚠️ Could not create bins for {col}")

#     return df_fe

# print("🔧 Performing advanced feature engineering...")
# train_fe = advanced_feature_engineering(train_data, is_train=True)
# test_fe = advanced_feature_engineering(test_data, is_train=False)

# # -----------------------------
# # Step 3: Smart Target Transformation
# # -----------------------------
# # Analyze target distribution
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 3, 1)
# plt.hist(train_data['purchaseValue'], bins=50, alpha=0.7)
# plt.title('Original Target')
# plt.subplot(1, 3, 2)
# plt.hist(np.log1p(train_data['purchaseValue']), bins=50, alpha=0.7)
# plt.title('Log1p Target')
# plt.subplot(1, 3, 3)
# plt.hist(np.sqrt(train_data['purchaseValue']), bins=50, alpha=0.7)
# plt.title('Sqrt Target')
# plt.tight_layout()
# plt.show()

# # Use Box-Cox-like transformation
# y_original = train_data["purchaseValue"]

# # Try different transformations and pick the best
# transformations = {
#     'log1p': np.log1p(y_original),
#     'sqrt': np.sqrt(y_original),
#     'cbrt': np.cbrt(y_original),
#     'log1p_plus_sqrt': np.log1p(y_original) + 0.1 * np.sqrt(y_original)
# }

# # Choose log1p as it typically works well for purchase data
# y_transformed = transformations['log1p']
# print(f"🎯 Using log1p transformation for target")

# # -----------------------------
# # Step 4: Intelligent Preprocessing
# # -----------------------------
# # Drop ID/session/timestamp columns
# drop_cols = ["userId", "sessionId", "date", "sessionStart"]
# X = train_fe.drop(columns=["purchaseValue"] + drop_cols, errors='ignore')
# y = y_transformed

# # Ensure test set has same columns (except target)
# test_X = test_fe.drop(columns=drop_cols, errors='ignore')

# # Align columns between train and test
# common_cols = list(set(X.columns) & set(test_X.columns))
# X = X[common_cols]
# test_X = test_X[common_cols]

# print(f"📊 Final feature count: {len(common_cols)}")

# # Split data with stratification based on target ranges
# # Create target bins for stratification
# y_bins = pd.qcut(y, q=5, labels=False, duplicates='drop')
# X_train, X_val, y_train, y_val = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y_bins
# )

# # Identify feature types with proper type checking
# numeric_features = []
# categorical_features = []

# for col in X.columns:
#     if X[col].dtype in ['int64', 'float64']:
#         numeric_features.append(col)
#     elif X[col].dtype == 'object':
#         categorical_features.append(col)
#     else:
#         # Convert any remaining problematic types to string
#         X[col] = X[col].astype(str)
#         test_X[col] = test_X[col].astype(str)
#         categorical_features.append(col)

# print(f"📈 Features: {len(numeric_features)} numeric, {len(categorical_features)} categorical")

# # Additional data cleaning for categorical features
# print("🧹 Cleaning categorical data...")
# for col in categorical_features:
#     # Replace any remaining problematic values
#     X[col] = X[col].fillna('missing').astype(str)
#     test_X[col] = test_X[col].fillna('missing').astype(str)

#     # Handle any remaining boolean-like strings
#     X[col] = X[col].replace({'True': 'true', 'False': 'false'})
#     test_X[col] = test_X[col].replace({'True': 'true', 'False': 'false'})

# # -----------------------------
# # Step 5: Advanced Preprocessing Pipeline
# # -----------------------------
# # Enhanced numeric preprocessing
# numeric_transformer = Pipeline([
#     ("imputer", SimpleImputer(strategy="median")),  # Median is more robust
#     ("scaler", StandardScaler())
# ])

# # Enhanced categorical preprocessing with better error handling
# def get_categorical_transformer():
#     return Pipeline([
#         ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
#         ("encoder", OneHotEncoder(
#             handle_unknown="ignore",
#             sparse_output=False,
#             max_categories=15,  # Reduce to prevent too many features
#             drop='if_binary'    # Drop one category for binary features
#         ))
#     ])

# categorical_transformer = get_categorical_transformer()

# preprocessor = ColumnTransformer([
#     ("num", numeric_transformer, numeric_features),
#     ("cat", categorical_transformer, categorical_features)
# ], remainder='drop')

# # -----------------------------
# # Step 6: Feature Selection with Multiple Methods
# # -----------------------------
# print("🔍 Performing intelligent feature selection...")

# # First, fit RandomForest for feature importance
# rf_temp = Pipeline([
#     ("preprocessor", preprocessor),
#     ("regressor", RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
# ])
# rf_temp.fit(X_train, y_train)

# # Get feature importance
# feature_names = rf_temp.named_steps["preprocessor"].get_feature_names_out()
# importances = rf_temp.named_steps["regressor"].feature_importances_

# # Create feature importance dataframe
# feat_df = pd.DataFrame({"feature": feature_names, "importance": importances})
# feat_df = feat_df.sort_values(by="importance", ascending=False)

# # Dynamic threshold based on cumulative importance
# cumsum_importance = feat_df['importance'].cumsum()
# threshold_idx = np.where(cumsum_importance >= 0.95)[0][0]  # Keep features that explain 95% of importance
# dynamic_threshold = feat_df.iloc[threshold_idx]['importance']

# print(f"📊 Dynamic importance threshold: {dynamic_threshold:.6f}")
# print(f"🎯 Keeping {threshold_idx + 1} features out of {len(feature_names)}")

# # Select important features
# important_features = feat_df.head(threshold_idx + 1)["feature"].tolist()

# # Map back to original features
# original_feature_names = []
# for feat in important_features:
#     if "__" in feat:
#         original_name = feat.split("__")[1]
#     else:
#         original_name = feat.replace("num__", "")

#     if original_name not in original_feature_names:
#         original_feature_names.append(original_name)

# # Safety check
# if len(original_feature_names) == 0:
#     print("⚠️ Using all features as fallback")
#     original_feature_names = numeric_features + categorical_features

# reduced_numeric = [f for f in original_feature_names if f in numeric_features]
# reduced_categorical = [f for f in original_feature_names if f in categorical_features]

# print(f"✅ Selected: {len(reduced_numeric)} numeric, {len(reduced_categorical)} categorical")

# # -----------------------------
# # Step 7: Advanced Model Ensemble
# # -----------------------------
# # Create reduced preprocessor
# reduced_preprocessor = ColumnTransformer([
#     ("num", numeric_transformer, reduced_numeric),
#     ("cat", categorical_transformer, reduced_categorical)
# ])

# # Define multiple models for ensemble
# models = {
#     'xgb': XGBRegressor(
#         objective="reg:squarederror",
#         n_estimators=500,
#         max_depth=6,
#         learning_rate=0.05,
#         subsample=0.8,
#         colsample_bytree=0.8,
#         random_state=42,
#         n_jobs=-1
#     ),
#     'lgb': LGBMRegressor(
#         objective='regression',
#         n_estimators=500,
#         max_depth=6,
#         learning_rate=0.05,
#         subsample=0.8,
#         colsample_bytree=0.8,
#         random_state=42,
#         n_jobs=-1,
#         verbose=-1
#     ),
#     'rf': RandomForestRegressor(
#         n_estimators=300,
#         max_depth=10,
#         min_samples_split=5,
#         min_samples_leaf=2,
#         random_state=42,
#         n_jobs=-1
#     ),
#     'gb': GradientBoostingRegressor(
#         n_estimators=300,
#         max_depth=6,
#         learning_rate=0.05,
#         subsample=0.8,
#         random_state=42
#     )
# }

# # Train individual models and collect predictions
# print("🚀 Training ensemble models...")
# model_predictions = {}
# model_scores = {}

# for name, model in models.items():
#     print(f"   Training {name.upper()}...")

#     pipeline = Pipeline([
#         ("preprocessor", reduced_preprocessor),
#         ("regressor", model)
#     ])

#     # Fit model
#     pipeline.fit(X_train, y_train)

#     # Predict on validation
#     y_val_pred = pipeline.predict(X_val)
#     r2 = r2_score(y_val, y_val_pred)

#     model_predictions[name] = y_val_pred
#     model_scores[name] = r2

#     print(f"   {name.upper()} R² Score: {r2:.4f}")

# # -----------------------------
# # Step 8: Optimal Ensemble Weighting
# # -----------------------------
# print("🎯 Finding optimal ensemble weights...")

# # Simple average ensemble
# ensemble_pred = np.mean([pred for pred in model_predictions.values()], axis=0)
# ensemble_r2 = r2_score(y_val, ensemble_pred)
# print(f"📊 Simple Average Ensemble R² Score: {ensemble_r2:.4f}")

# # Weighted ensemble based on individual performance
# weights = np.array([model_scores[name] for name in models.keys()])
# weights = weights / weights.sum()  # Normalize

# weighted_ensemble_pred = np.average(
#     [model_predictions[name] for name in models.keys()],
#     axis=0,
#     weights=weights
# )
# weighted_ensemble_r2 = r2_score(y_val, weighted_ensemble_pred)
# print(f"⚖️ Weighted Ensemble R² Score: {weighted_ensemble_r2:.4f}")

# # Choose best ensemble method
# if weighted_ensemble_r2 > ensemble_r2:
#     final_ensemble_pred = weighted_ensemble_pred
#     final_r2 = weighted_ensemble_r2
#     print("✅ Using weighted ensemble")
# else:
#     final_ensemble_pred = ensemble_pred
#     final_r2 = ensemble_r2
#     print("✅ Using simple average ensemble")

# # -----------------------------
# # Step 9: Final Model Training on Full Data
# # -----------------------------
# print("🏁 Training final models on full training data...")

# # Retrain all models on full training data
# final_models = {}
# for name, model in models.items():
#     pipeline = Pipeline([
#         ("preprocessor", reduced_preprocessor),
#         ("regressor", model)
#     ])
#     pipeline.fit(X, y)
#     final_models[name] = pipeline

# # -----------------------------
# # Step 10: Test Predictions
# # -----------------------------
# print("🔮 Generating test predictions...")

# # Generate predictions from all models
# test_predictions = {}
# for name, model in final_models.items():
#     test_pred_log = model.predict(test_X)
#     test_predictions[name] = test_pred_log

# # Create ensemble prediction
# if weighted_ensemble_r2 > ensemble_r2:
#     final_test_pred_log = np.average(
#         [test_predictions[name] for name in models.keys()],
#         axis=0,
#         weights=weights
#     )
# else:
#     final_test_pred_log = np.mean([pred for pred in test_predictions.values()], axis=0)

# # Transform back to original scale
# final_test_pred = np.expm1(final_test_pred_log)

# # Ensure no negative predictions
# final_test_pred = np.maximum(final_test_pred, 0)

# # -----------------------------
# # Step 11: Create Submission
# # -----------------------------
# submission = pd.DataFrame({
#     "id": range(len(final_test_pred)),
#     "purchaseValue": final_test_pred
# })

# submission.to_csv("submission.csv", index=False)
# print("📁 submission.csv saved successfully!")

# # -----------------------------
# # Step 12: Model Analysis and Insights
# # -----------------------------
# print("\n🏆 Model Performance Summary:")
# print("=" * 50)
# for name, score in model_scores.items():
#     print(f"{name.upper():>12}: {score:.4f}")
# print(f"{'ENSEMBLE':>12}: {final_r2:.4f}")
# print("=" * 50)

# print(f"\n📈 Final Validation R² Score: {final_r2:.4f}")
# print(f"📊 Prediction range: ${final_test_pred.min():.2f} - ${final_test_pred.max():.2f}")
# print(f"💰 Mean predicted value: ${final_test_pred.mean():.2f}")

# # Show top features
# print(f"\n🔍 Top 15 Most Important Features:")
# print("-" * 50)
# for i, (_, row) in enumerate(feat_df.head(15).iterrows()):
#     feature_name = row['feature']
#     if "__" in feature_name:
#         display_name = feature_name.split("__")[1]
#     else:
#         display_name = feature_name.replace("num__", "")
#     print(f"{i+1:>2}. {display_name:<30} ({row['importance']:.4f})")

# print("\n✅ Enhanced model training completed!")
# print("🎯 This model includes:")
# print("   • Advanced feature engineering")
# print("   • Multiple transformation strategies")
# print("   • Intelligent feature selection")
# print("   • Ensemble of 4 different algorithms")
# print("   • Optimal weight calculation")
# print("   • Robust preprocessing pipeline")

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import Ridge
# from sklearn.metrics import r2_score
# import pandas as pd
# import numpy as np

# # Load your data (adjust file paths as needed)
# print("Loading data...")
# train_data = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv')
# test_data = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv')
# print(f"Train data shape: {train_data.shape}")
# print(f"Test data shape: {test_data.shape}")
# print(f"Train columns: {list(train_data.columns)}")
# print(f"Test columns: {list(test_data.columns)}")

# # Enhanced feature engineering function
# def create_features(df):
#     df = df.copy()

#     # Age-based features
#     if 'age' in df.columns:
#         df['age_squared'] = df['age'] ** 2
#         df['age_bins'] = pd.cut(df['age'], bins=[0, 25, 35, 50, 65, 100], labels=['young', 'adult', 'middle', 'senior', 'elderly'])

#     # Income-based features
#     if 'income' in df.columns:
#         df['income_log'] = np.log1p(df['income'])  # log transform to handle skewness
#         df['income_bins'] = pd.qcut(df['income'], q=5, labels=['low', 'below_avg', 'avg', 'above_avg', 'high'], duplicates='drop')

#     # Interaction features (if both exist)
#     if 'age' in df.columns and 'income' in df.columns:
#         df['age_income_ratio'] = df['age'] / (df['income'] + 1)
#         df['income_per_age'] = df['income'] / (df['age'] + 1)

#     # Family size features
#     if 'familySize' in df.columns:
#         df['is_single'] = (df['familySize'] == 1).astype(int)
#         df['large_family'] = (df['familySize'] >= 5).astype(int)

#     return df

# # Apply feature engineering
# print("Creating enhanced features...")
# X_enhanced = create_features(train_data.drop(columns=['purchaseValue']))
# y = train_data['purchaseValue']

# # Identify feature types
# numeric_features = X_enhanced.select_dtypes(include=['int64', 'float64']).columns
# categorical_features = X_enhanced.select_dtypes(include=['object', 'category']).columns

# print(f"Numeric features: {len(numeric_features)}")
# print(f"Categorical features: {len(categorical_features)}")

# # Preprocessing pipelines
# numeric_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='median')),  # Median is more robust to outliers
#     ('scaler', StandardScaler())
# ])

# categorical_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
# ])

# # Combine preprocessing
# preprocessor = ColumnTransformer([
#     ('num', numeric_pipeline, numeric_features),
#     ('cat', categorical_pipeline, categorical_features)
# ])

# # Split data
# X_train, X_val, y_train, y_val = train_test_split(X_enhanced, y, test_size=0.2, random_state=42, stratify=pd.qcut(y, q=5, duplicates='drop'))

# print("Training models...")

# # Model 1: Optimized Random Forest (faster parameters)
# rf_model = Pipeline([
#     ('preprocessor', preprocessor),
#     ('regressor', RandomForestRegressor(
#         n_estimators=100,        # Reasonable number
#         max_depth=15,           # Prevent overfitting
#         min_samples_split=5,    # Faster training
#         min_samples_leaf=2,     # Prevent overfitting
#         max_features='sqrt',    # Faster training
#         random_state=42,
#         n_jobs=-1               # Use all cores
#     ))
# ])

# # Model 2: Ridge Regression (very fast)
# ridge_model = Pipeline([
#     ('preprocessor', preprocessor),
#     ('regressor', Ridge(alpha=1.0, random_state=42))
# ])

# # Train models
# print("Training Random Forest...")
# rf_model.fit(X_train, y_train)

# print("Training Ridge Regression...")
# ridge_model.fit(X_train, y_train)

# # Evaluate models
# rf_pred = rf_model.predict(X_val)
# ridge_pred = ridge_model.predict(X_val)

# rf_r2 = r2_score(y_val, rf_pred)
# ridge_r2 = r2_score(y_val, ridge_pred)

# print(f'Random Forest R² Score: {rf_r2:.4f}')
# print(f'Ridge Regression R² Score: {ridge_r2:.4f}')

# # Ensemble approach - simple average
# ensemble_pred = (rf_pred + ridge_pred) / 2
# ensemble_r2 = r2_score(y_val, ensemble_pred)
# print(f'Ensemble R² Score: {ensemble_r2:.4f}')

# # Choose best model
# models = {'RF': (rf_model, rf_r2), 'Ridge': (ridge_model, ridge_r2)}
# best_single_name, (best_single_model, best_single_r2) = max(models.items(), key=lambda x: x[1][1])

# if ensemble_r2 > best_single_r2:
#     print(f'Best model: Ensemble (R² = {ensemble_r2:.4f})')
#     use_ensemble = True
# else:
#     print(f'Best model: {best_single_name} (R² = {best_single_r2:.4f})')
#     use_ensemble = False

# # Prepare test data
# print("Preparing test predictions...")
# X_test_enhanced = create_features(test_data)

# # Make predictions
# if use_ensemble:
#     rf_test_pred = rf_model.predict(X_test_enhanced)
#     ridge_test_pred = ridge_model.predict(X_test_enhanced)
#     test_predictions = (rf_test_pred + ridge_test_pred) / 2
# else:
#     test_predictions = best_single_model.predict(X_test_enhanced)

# # Create submission
# submission = pd.DataFrame({
#     'id': range(0, test_data.shape[0]),
#     'purchaseValue': test_predictions
# })

# submission.to_csv('submission.csv', index=False)
# print("Submission file saved as 'submission.csv'")

# # Feature importance (if Random Forest is best)
# if not use_ensemble and best_single_name == 'RF':
#     feature_names = (list(numeric_features) +
#                     list(rf_model.named_steps['preprocessor']
#                          .named_transformers_['cat']
#                          .named_steps['encoder']
#                          .get_feature_names_out(categorical_features)))

#     importances = rf_model.named_steps['regressor'].feature_importances_
#     feature_importance = pd.DataFrame({
#         'feature': feature_names,
#         'importance': importances
#     }).sort_values('importance', ascending=False)

#     print("\nTop 10 Most Important Features:")
#     print(feature_importance.head(10))

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import r2_score
# from lightgbm import LGBMRegressor

# # =======================
# # 1. Load the data
# # =======================
# train_data = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv')
# test_data = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv')

# # =======================
# # 2. Feature/target split
# # =======================
# X = train_data.drop(columns=['purchaseValue'])
# y = train_data['purchaseValue']

# # =======================
# # 3. Preprocessing
# # =======================
# numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
# categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# numeric_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# categorical_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
# ])

# preprocessor = ColumnTransformer([
#     ('num', numeric_pipeline, numeric_features),
#     ('cat', categorical_pipeline, categorical_features)
# ])

# # =======================
# # 4. Train/validation split
# # =======================
# X_train, X_val, y_train, y_val = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# # =======================
# # 5. Model pipeline
# # =======================
# model_pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('regressor', LGBMRegressor(random_state=42))
# ])

# # =======================
# # 6. Hyperparameter tuning
# # =======================
# param_distributions = {
#     'regressor__n_estimators': [100, 200, 300],
#     'regressor__max_depth': [3, 6, 10],
#     'regressor__learning_rate': [0.01, 0.1, 0.2],
#     'regressor__subsample': [0.7, 0.8, 1.0],
#     'regressor__colsample_bytree': [0.7, 0.8, 1.0]
# }

# random_search = RandomizedSearchCV(
#     model_pipeline,
#     param_distributions=param_distributions,
#     n_iter=20,
#     cv=3,
#     scoring='r2',
#     n_jobs=-1,
#     verbose=1,
#     random_state=42
# )

# random_search.fit(X_train, y_train)

# # =======================
# # 7. Validation
# # =======================
# best_model = random_search.best_estimator_
# y_pred = best_model.predict(X_val)
# r2 = r2_score(y_val, y_pred)
# print(f'Validation R2 Score: {r2:.4f}')

# # =======================
# # 8. Test prediction & submission
# # =======================
# X_test = test_data  # Ensure test_data has the same columns as X
# test_predictions = best_model.predict(X_test)

# submission = pd.DataFrame({
#     'id': range(len(test_predictions)),
#     'purchaseValue': test_predictions
# })
# submission.to_csv('submission.csv', index=False)
# print("Submission file 'submission.csv' created successfully.")

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import r2_score
# from lightgbm import LGBMRegressor

# # =======================
# # 1. Load the data from URLs
# # =======================

# train_data = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv')
# test_data = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv')

# # =======================
# # 2. Feature Engineering (example: add more if you can)
# # =======================
# # Example: create interaction feature (customize based on your columns)
# if 'device' in train_data.columns and 'browser' in train_data.columns:
#     train_data['device_browser'] = train_data['device'] + '_' + train_data['browser']
#     test_data['device_browser'] = test_data['device'] + '_' + test_data['browser']

# # =======================
# # 3. Prepare features and target
# # =======================
# X = train_data.drop(columns=['purchaseValue'])
# y = train_data['purchaseValue']
# X_test = test_data.copy()

# # Optional: Log-transform target if highly skewed
# # y = np.log1p(y)

# # =======================
# # 4. Preprocessing
# # =======================
# numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
# categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# numeric_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# categorical_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
# ])

# preprocessor = ColumnTransformer([
#     ('num', numeric_pipeline, numeric_features),
#     ('cat', categorical_pipeline, categorical_features)
# ])

# # =======================
# # 5. Model and Hyperparameter Tuning
# # =======================
# model = Pipeline([
#     ('preprocessor', preprocessor),
#     ('regressor', LGBMRegressor(random_state=42))
# ])

# param_distributions = {
#     'regressor__n_estimators': [200, 300, 500],
#     'regressor__max_depth': [6, 10, 15],
#     'regressor__learning_rate': [0.01, 0.05, 0.1],
#     'regressor__num_leaves': [31, 50, 100],
#     'regressor__reg_alpha': [0, 0.1, 1],
#     'regressor__reg_lambda': [0, 0.1, 1],
#     'regressor__subsample': [0.7, 0.8, 1.0],
#     'regressor__colsample_bytree': [0.7, 0.8, 1.0]
# }

# # K-Fold Cross Validation
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# random_search = RandomizedSearchCV(
#     model,
#     param_distributions=param_distributions,
#     n_iter=20,
#     cv=kf,
#     scoring='r2',
#     n_jobs=-1,
#     verbose=1,
#     random_state=42
# )

# random_search.fit(X, y)
# print(f"Best R2 from CV: {random_search.best_score_:.4f}")

# # =======================
# # 6. Predict on Test Set
# # =======================
# best_model = random_search.best_estimator_
# test_predictions = best_model.predict(X_test)

# # If you log-transformed y earlier, reverse it here:
# # test_predictions = np.expm1(test_predictions)

# # =======================
# # 7. Prepare Submission
# # =======================
# # If your test set has an 'id' column, use it. Else, use a range.
# if 'id' in test_data.columns:
#     submission = pd.DataFrame({'id': test_data['id'], 'purchaseValue': test_predictions})
# else:
#     submission = pd.DataFrame({'id': range(len(test_predictions)), 'purchaseValue': test_predictions})

# submission.to_csv('submission.csv', index=False)
# print("Submission file 'submission.csv' created successfully.")


import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

# =======================
# 1. Load Data
# =======================
train_data = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv')
test_data = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv')

# =======================
# 2. Feature Selection & Engineering
# =======================
# Drop columns that are IDs, session identifiers, or not available at prediction time
drop_cols = [
    'purchaseValue', 'userId', 'sessionId', 'date', 'sessionStart', 'gclIdPresent'
]
drop_cols = [col for col in drop_cols if col in train_data.columns]

# Convert device.isMobile to integer if present
for df in [train_data, test_data]:
    if 'device.isMobile' in df.columns:
        df['isMobile'] = df['device.isMobile'].map({True:1, False:0, 'TRUE':1, 'FALSE':0, np.nan:0})

# Prepare features and target
y = np.log1p(train_data['purchaseValue'])
X = train_data.drop(columns=drop_cols)
X_test = test_data.drop(columns=[col for col in drop_cols if col != 'purchaseValue'])

# Identify categorical and numeric columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()

# =======================
# 3. Preprocessing Pipeline
# =======================
preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numeric_features),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]), categorical_features)
])

# =======================
# 4. Model
# =======================
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(
        n_estimators=800,
        max_depth=8,
        learning_rate=0.03,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=42,
        n_jobs=1  # Set to 1 to avoid memory issues
    ))
])

# =======================
# 5. Cross-Validation (Optional, can skip for speed)
# =======================
kf = KFold(n_splits=3, shuffle=True, random_state=42)
r2_scores = []
for train_idx, val_idx in kf.split(X):
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
    model.fit(X_tr, y_tr)
    preds = model.predict(X_val)
    score = r2_score(np.expm1(y_val), np.expm1(preds))
    r2_scores.append(score)
    print(f"Fold R²: {score:.4f}")
print(f"Mean CV R²: {np.mean(r2_scores):.4f}")

# =======================
# 6. Train on Full Data & Predict
# =======================
model.fit(X, y)
test_pred = np.expm1(model.predict(X_test))

# =======================
# 7. Submission
# =======================
id_col = 'userId' if 'userId' in test_data.columns else test_data.index
submission = pd.DataFrame({
    'id': range(0, test_data.shape[0]),
    'purchaseValue': test_pred
})
submission.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' created successfully.")
